In [4]:
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.passes import FullAncillaAllocation, EnlargeWithAncilla, ApplyLayout
from utils import file_to_coupling_map, directory_to_circuits

import random
import time
import os
import numpy as np


In [5]:
seed = 42
random.seed(seed)

cmap_names = ["toyko_20.txt"]
qc_name = "paper"
heuristic_list = ["lookahead"]
qc_list = directory_to_circuits(f"circuits/{qc_name}")
num_runs = 5
seed_list = [random.randint(0, 100000) for _ in range(num_runs * len(qc_list))]
pass_trials = 20

# CSV file to store the results
csv_path = f"results/{qc_name}_new.csv"
# Columns are: "qc_label, swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
with open(csv_path, "w") as f:
    f.write("qc_label,swap_count,depth,runtime,num_qubits,heuristic,cmap,trials,seed\n")

num_runs = 5




In [6]:
# Run on all heuristics
for heuristic in heuristic_list:
    print(f"Running {heuristic} heuristic")
    # Run on all coupling maps
    for cmap_name in cmap_names: 
        print(f"    Loading coupling map {cmap_name}")
        cmap = file_to_coupling_map(f"coupling_maps/{cmap_name}")
        # Run on all circuits
        for qc in qc_list:
            qc_label = qc.name
            num_qubits = qc.num_qubits
            for i in range(num_runs):
                curr_seed = seed_list.pop()
                print(f"        Running {qc.name} on {cmap_name} with seed {curr_seed}")
                # Create the routing pass
                rp = SabreSwap(coupling_map=cmap, heuristic=heuristic, seed=curr_seed, trials=pass_trials)
                lp = SabreLayout(coupling_map=cmap, routing_pass=rp, seed=curr_seed)
                # Create the pass manager
                pm = PassManager([lp, FullAncillaAllocation(cmap), EnlargeWithAncilla(), ApplyLayout(), rp])
                # Execute the pass manager
                start = time.time()
                qc_tr = pm.run(qc)
                end = time.time()
                # Collect the metrics for this trial
                runtime = end - start
                swap_count = qc_tr.count_ops().get("swap", 0)
                depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2)

                # Write the results to the CSV file
                # Columns are: "qc_label, swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
                with open(csv_path, "a") as f:
                    f.write(f"{qc_label},{swap_count},{depth},{runtime},{num_qubits},{heuristic},{cmap_name},{pass_trials},{curr_seed}\n")
                print(f"            Swap count: {swap_count}, Depth: {depth}, Runtime: {runtime}")

    

Running lookahead heuristic
    Loading coupling map toyko_20.txt
        Running qft_10.qasm on toyko_20.txt with seed 28785
            Swap count: 22, Depth: 53, Runtime: 0.1288292407989502
        Running qft_10.qasm on toyko_20.txt with seed 73000
            Swap count: 19, Depth: 47, Runtime: 0.1193380355834961
        Running qft_10.qasm on toyko_20.txt with seed 90198
            Swap count: 27, Depth: 44, Runtime: 0.16370701789855957
        Running qft_10.qasm on toyko_20.txt with seed 83886
            Swap count: 22, Depth: 46, Runtime: 0.10021400451660156
        Running qft_10.qasm on toyko_20.txt with seed 35382
            Swap count: 17, Depth: 40, Runtime: 0.08841800689697266
        Running qft_13.qasm on toyko_20.txt with seed 49735
            Swap count: 40, Depth: 79, Runtime: 0.13744187355041504
        Running qft_13.qasm on toyko_20.txt with seed 60589
            Swap count: 40, Depth: 75, Runtime: 0.16534113883972168
        Running qft_13.qasm on toyko_20.